In [29]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [30]:
print("bw2data version", bd.__version__)
print("bw2io version", bi.__version__)
print("bw2calc version", bi.__version__)

bw2data version 4.5.3
bw2io version 0.9.11
bw2calc version 0.9.11


In [31]:
bd.projects


Brightway2 projects manager with 8 objects:
	Another project
	Bicycle example
	USEEIO-1.1
	default
	ecoinvent-3.11-cutoff-bw25
	ecoinvent-3.12-cutoff
	electric_vehicle_standalone
	manure
Use `projects.report()` to get a report on all projects.

In [32]:
bd.projects.delete_project("manure", delete_dir=True)
bd.projects

Brightway2 projects manager with 7 objects:
	Another project
	Bicycle example
	USEEIO-1.1
	default
	ecoinvent-3.11-cutoff-bw25
	ecoinvent-3.12-cutoff
	electric_vehicle_standalone
Use `projects.report()` to get a report on all projects.

In [33]:
if "manure" in bd.projects:
    bd.projects.set_current("manure")
elif "ecoinvent-3.11-cutoff-bw25" in bd.projects:
    bd.projects.set_current("ecoinvent-3.11-cutoff-bw25")
    bd.projects.copy_project("manure", switch=True)
else:
    print("A manure project doesn't exist and a copy-able ecoinvent is also not found.")

print("active project:", bd.projects.current)

active project: manure


In [34]:
bd.projects
#to confirm that the new project has indeed been created

Brightway2 projects manager with 8 objects:
	Another project
	Bicycle example
	USEEIO-1.1
	default
	ecoinvent-3.11-cutoff-bw25
	ecoinvent-3.12-cutoff
	electric_vehicle_standalone
	manure
Use `projects.report()` to get a report on all projects.

In [35]:
db_name = "foreground_db"
if db_name not in bd.databases:
    bd.Database(db_name).register()
db = bd.Database(db_name)

db_ei = bd.Database("ecoinvent-3.11-cutoff")

In [36]:
bd.databases
#to see if any database has been imported already

Databases dictionary with 3 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff
	foreground_db

In [37]:
# Load the biosphere database
b_bio = bd.Database("ecoinvent-3.11-biosphere")

# Search for methane or CH4 (case-insensitive)
for flow in b_bio:
    if "methane, non-fossil" in flow["name"].lower() or "ch4" in flow["name"].lower() and act.get("categories", "").lower():
        print(f"{flow['name']} | {flow.get('categories', '')} | {flow.get('code', '')}")


flow.as_dict()

Methane, non-fossil | ('air', 'urban air close to ground') | baf58fc9-573c-419c-8c16-831ac03203b9
Methane, non-fossil | ('air', 'non-urban air or from high stacks') | 57bdb443-d4a6-423d-8024-959b8261d02e
Methane, non-fossil | ('air',) | da1157e2-7593-4dfd-80dd-a3449b37a4d8
Methane, non-fossil | ('air', 'low population density, long-term') | 6c977009-5c4e-4901-a4c1-ab20389cb972


{'categories': ('soil', 'agricultural'),
 'code': 'cd08eddd-d946-40cf-b0b5-87f6dbd7eb21',
 'CAS number': '000063-25-2',
 'synonyms': ['Sevin', 'naphthalen-1-yl N-methylcarbamate'],
 'name': 'Carbaryl',
 'database': 'ecoinvent-3.11-biosphere',
 'unit': 'kilogram',
 'type': 'emission',
 'id': 246559627108327485}

In [38]:

b_ei = bd.Database("ecoinvent-3.11-cutoff")

# Search for electricity activities in Denmark (location code likely "DNK")
for act in b_ei:
    if "electricity, medium" in act["name"].lower() and act.get("location", "").lower() == "dk":
        print(f"{act['name']} | {act.get('location', '')} | {act.get('code', '')}")



electricity, medium voltage, residual mix | DK | 7274eddfd02218733d7349c975291423
electricity, from municipal waste incineration to generic market for electricity, medium voltage | DK | 6bc2f531645dc36e3bb7ee0a24941222
market for electricity, medium voltage | DK | 6bcbd7fc891c8b3bf0dd96027c1f9b0e


In [39]:
# 3. Get the ecoinvent electricity activity (Switzerland)
ei_elec = b_ei.get("6bcbd7fc891c8b3bf0dd96027c1f9b0e")
print(f"✅ Using electricity process: {ei_elec['name']} | {ei_elec['location']}")



✅ Using electricity process: market for electricity, medium voltage | DK


In [40]:
#Creating the various nodes / processes in the project

manure_production = db.new_node(
    code="manure_production",
    name='manure_production',
    location="DK",
    type=bd.labels.process_node_default,
)
manure_production.save()


manure = db.new_node(
    code="manure",
    name='manure',
    unit="kg",
    type=bd.labels.product_node_default,
)
manure.save()


dewatering = db.new_node(
    code="dewatering",
    name='dewatering',
    location='DK',
    type=bd.labels.process_node_default,
)
dewatering.save()

#we don't add a electricity product because we simply call it directly as an edge from ecoinvent

 
dewatering_product = db.new_node(
    code = "dewatering_product",
    name = 'dewatering_product',
    unit="kg",
    type=bd.labels.product_node_default,
) 
dewatering_product.save()



soil_application = db.new_node(
    code = "soil_application",
    name = 'soil_application',
    unit="kg",
    type=bd.labels.process_node_default,
) 
soil_application.save()


storage = db.new_node(
    code = "storage",
    name = 'storage',
    unit="kg",
    type=bd.labels.process_node_default,
) 
storage.save()


soil_application_product = db.new_node(
    code = "soil_application_product",
    name = 'soil_application_product',
    unit="kg",
    type=bd.labels.product_node_default,
) 
soil_application_product.save()


storage_product = db.new_node(
    code = "storage_product",
    name = 'storage_product',
    unit="kg",
    type=bd.labels.product_node_default,
) 
storage_product.save()


manure_mgmt = db.new_node(
    code = "manure_mgmt",
    name = 'manure_mgmt',
    unit="kg",
    type=bd.labels.process_node_default,
) 
manure_mgmt.save()


manure_managed = db.new_node(
    code = "manure_managed",
    name = 'manure_managed',
    unit="kg",
    type=bd.labels.product_node_default,
) 
manure_managed.save()




In [41]:
ei_methane = b_bio.get("da1157e2-7593-4dfd-80dd-a3449b37a4d8")
print(f"✅ Using electricity process: {ei_methane['name']} | {ei_methane['categories']}")


✅ Using electricity process: Methane, non-fossil | ('air',)


In [42]:
#I am now creating a cell to retrieve automatically the amount data from the CSV file created. 

# Load the CSV
file_path = "data/output_mfa.csv"
mfa_data = pd.read_csv(file_path)

# Print the entire content
print(mfa_data)


                        flow_name   substance  \
0            sysenv => dewatering  dry matter   
1            sysenv => dewatering       water   
2           dewatering => storage  dry matter   
3           dewatering => storage       water   
4  dewatering => soil application  dry matter   
5  dewatering => soil application       water   
6               storage => sysenv  dry matter   
7               storage => sysenv       water   
8      soil application => sysenv  dry matter   
9      soil application => sysenv       water   

                                     flow_id  value  
0            sysenv => dewatering_dry matter  100.0  
1                 sysenv => dewatering_water  900.0  
2           dewatering => storage_dry matter   40.0  
3                dewatering => storage_water  720.0  
4  dewatering => soil application_dry matter   60.0  
5       dewatering => soil application_water  180.0  
6               storage => sysenv_dry matter   40.0  
7                    storage

In [43]:
# Helper function to get amount from CSV
def get_amount(flow_pattern, substance=None):
    if substance:
        # Filter for a specific substance
        row = mfa_data[(mfa_data['flow_name'].str.contains(flow_pattern)) & (mfa_data['substance'] == substance)]
    else:
        # Sum all substances for the flow pattern
        row = mfa_data[mfa_data['flow_name'].str.contains(flow_pattern)]
    return row['value'].sum()

In [44]:
manure_production.new_edge(
    #amount=amount_manure,  #to be linked directly to the MFA
    amount=get_amount("sysenv => dewatering"),  # Sum of dry matter + water = 1000kg
    location= "DK",   #to be linked directly to the MFA
    name="manure_1",
    unit = "kg",
    input=manure,
    type=bd.labels.production_edge_default,
    functional=False,
).save()


dewatering.new_edge(
    #amount=amount_manure,  #to be linked directly to the MFA
    amount=get_amount("sysenv => dewatering"),  # Sum of dry matter + water = 1000kg
    location= "DK",   #to be linked directly to the MFA
    name="manure_2",
    unit = "kg",
    input=manure,
    type=bd.labels.consumption_edge_default,
    functional=False,
).save()


dewatering.new_edge(
    #amount=amount_manure,  #to be linked directly to the MFA
    amount=get_amount("dewatering => soil application") + get_amount("dewatering => storage"),  # Sum of dry matter + water = 1000kg
    location= "DK",   #to be linked directly to the MFA
    name="dprod_1",
    unit = "kg",
    input=dewatering_product,
    type=bd.labels.production_edge_default,
    functional=False,
).save()


dewatering.new_edge(
    name = "ei_elect",
    location = "DK", 
    amount = 1.1 * get_amount("sysenv => dewatering", "water"),
    unit = "kwh",
    input=ei_elec, #calling directly the selected ecoinvent process
    type=bd.labels.consumption_edge_default,
).save()


soil_application.new_edge(
    name= "fibres_1",
    #amount=amount_fibres,
    amount=get_amount("dewatering => soil application"),
    unit = "kg",
    location= "DK",
    input=dewatering_product,
    type=bd.labels.consumption_edge_default,
).save()


soil_application.new_edge(
    name= "fibres_2",
    #amount=amount_fibres,
    amount=get_amount("dewatering => soil application"),
    unit = "kg",
    location= "DK",
    input=soil_application_product,
    type=bd.labels.production_edge_default,
).save()


storage.new_edge(
    name= "liquid_1",
    #amount=amount_liquid,
    amount=get_amount("dewatering => storage"),
    unit = "kg",
    location= "DK",
    input=dewatering_product,
    type=bd.labels.consumption_edge_default,
).save()


storage.new_edge(
    name= "liquid_2",
    #amount=amount_liquid,
    amount=get_amount("dewatering => storage"),
    unit = "kg",
    location= "DK",
    input=storage_product,
    type=bd.labels.production_edge_default,
).save()


storage.new_edge(
    name = "methane",
    location = "DK",
    amount= get_amount("dewatering => storage") * 0.5, #expressed as 0.5 * liquid_1
    unit = "kg",
    input=ei_methane,
    type=bd.labels.biosphere_edge_default,
).save()

#
manure_mgmt.new_edge(
    name= "soil_prod",
    #amount=amount_liquid,
    amount=get_amount("dewatering => soil application"),
    location= "DK",
    input=soil_application_product,
    type=bd.labels.consumption_edge_default,
).save()


manure_mgmt.new_edge(
    name= "storage_prod",
    #amount=amount_fibres,
    amount=get_amount("dewatering => storage"),
    location= "DK",
    input=storage_product,
    type=bd.labels.consumption_edge_default,
).save()


manure_mgmt.new_edge(
    name= "managed_manure",
    #amount=1000,
    amount=get_amount("storage => sysenv") + get_amount("soil production => sysenv"),
    location= "DK",
    input=manure_managed,
    type=bd.labels.production_edge_default,
    functional=True,
).save()





In [45]:
for act in bd.Database("foreground_db"):
    print()
    print(act)
    for e in act.edges():
        print(e)


'manure_production' (None, DK, None)
Exchange: 1000.0 kg 'manure' (kg, GLO, None) from 'manure_production' (None, DK, None)

'storage_product' (kg, GLO, None)

'dewatering_product' (kg, GLO, None)

'manure' (kg, GLO, None)

'soil_application_product' (kg, GLO, None)

'manure_mgmt' (kg, GLO, None)
Exchange: 240.0 kg 'soil_application_product' (kg, GLO, None) to 'manure_mgmt' (kg, GLO, None)
Exchange: 760.0 kg 'storage_product' (kg, GLO, None) to 'manure_mgmt' (kg, GLO, None)
Exchange: 760.0 kg 'manure_managed' (kg, GLO, None) from 'manure_mgmt' (kg, GLO, None)

'manure_managed' (kg, GLO, None)

'soil_application' (kg, GLO, None)
Exchange: 240.0 kg 'dewatering_product' (kg, GLO, None) to 'soil_application' (kg, GLO, None)
Exchange: 240.0 kg 'soil_application_product' (kg, GLO, None) from 'soil_application' (kg, GLO, None)

'storage' (kg, GLO, None)
Exchange: 760.0 kg 'dewatering_product' (kg, GLO, None) to 'storage' (kg, GLO, None)
Exchange: 760.0 kg 'storage_product' (kg, GLO, None) fr

In [63]:
[e for e in storage.biosphere()][0]["input"]

('ecoinvent-3.11-biosphere', 'da1157e2-7593-4dfd-80dd-a3449b37a4d8')

In [68]:
from IPython.display import Markdown

def bw_to_flowchart(database_name):
    output_mermaid = "flowchart LR"

    db = bd.Database(database_name)
        
    #production exchanges
    for node in db:
        # safe_name = node['name'].replace(" ", "_")
        for edge in [edge for edge in node.production()]:
            process_name = node['name'].replace(" ", "_")
            product_db = edge["input"][0]
            product_code = edge["input"][1]
            product_name = bd.get_node(database=product_db, code=product_code)["name"].replace(" ", "_")
            if product_db != database_name:
                product_name = product_name + "_" + product_db 
            output_mermaid = output_mermaid + "\n" + f"\t{process_name} --> {product_name}{{{product_name}}}"
    
        #consumption exchanges        
        for edge in [edge for edge in node.technosphere()]:
            if edge["type"] != "production":
                process_name = node['name'].replace(" ", "_")
                product_db = edge["input"][0]
                product_code = edge["input"][1]
                product_name = bd.get_node(database=product_db, code=product_code)["name"].replace(" ", "_")
                if product_db != database_name:
                    product_name = product_name + "_" + product_db
                output_mermaid = output_mermaid + "\n" + f"{product_name}{{{product_name}}} --> \t{process_name}"
    
        #biosphere exchanges 
        for edge in node.biosphere():
            product_db = edge["input"][0]
            product_code = edge["input"][1]
            biosphere_name = bd.get_node(database=product_db, code=product_code)["name"].replace(" ", "_")
            if product_db != database_name:
                biosphere_name = biosphere_name + "_" + product_db
            output_mermaid = output_mermaid + "\n" + f"\t{process_name} --> {biosphere_name}{{{biosphere_name}}}"
    
    return Markdown(f"```mermaid\n{output_mermaid}\n```")

bw_to_flowchart("foreground_db")

```mermaid
flowchart LR
	soil_application --> soil_application_product{soil_application_product}
dewatering_product{dewatering_product} --> 	soil_application
	dewatering --> dewatering_product{dewatering_product}
manure{manure} --> 	dewatering
market_for_electricity,_medium_voltage_ecoinvent-3.11-cutoff{market_for_electricity,_medium_voltage_ecoinvent-3.11-cutoff} --> 	dewatering
	manure_production --> manure{manure}
	manure_mgmt --> manure_managed{manure_managed}
soil_application_product{soil_application_product} --> 	manure_mgmt
storage_product{storage_product} --> 	manure_mgmt
	storage --> storage_product{storage_product}
dewatering_product{dewatering_product} --> 	storage
	storage --> Methane,_non-fossil_ecoinvent-3.11-biosphere{Methane,_non-fossil_ecoinvent-3.11-biosphere}
```

In [48]:
#sorted(bd.methods)

In [49]:
my_method = bd.Method(
    ('ecoinvent-3.11',
     'CML v4.8 2016',
     'climate change',
     'global warming potential (GWP100)'),
)

my_method.name

functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    demand={manure_managed: 1000},
    method=my_method.name,
)

my_lca = bc.LCA(demand=functional_unit, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()
my_lca.score

14232.296346234714

In [50]:
lca_score = my_lca.score
impact_unit = bd.Method(('ecoinvent-3.11', 'CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')).metadata['unit']

# Extract functional unit details
amount_fu = list(functional_unit.values())[0]
activity_id = list(functional_unit.keys())[0]
activity = bd.get_activity(activity_id)
fu_unit = activity.get('unit', 'units')  # Dynamically fetch the unit (e.g., 'kg', 'm3', etc.)
fu_name = activity.get('name', 'unknown')  # Dynamically fetch the process name

# Print the result
print(f"The GWP (100) impact for {amount_fu} {fu_unit} of {fu_name} is: {lca_score:.4g} {impact_unit}")


The GWP (100) impact for 1000 kg of manure_managed is: 1.423e+04 kg CO2-Eq
